In [1]:
import numpy as np
import plotly.graph_objects as go

def sinusoidal_synthetic(x: np.ndarray) -> np.ndarray:
    r"""
    Computes the function f(x) = -(x-1)^2 * \sin(3x + 5^{-1} + 1) for a given numpy input x.

    Args:
        x (np.ndarray): Input array of shape (N, 1) where N is the number of data points.
                        If the input is (N,), it will be automatically reshaped to (N, 1).

    Returns:
        np.ndarray: Output array of shape (N, 1) representing the computed values of f(x).

    f(x) = -(x-1)^2 \sin(3x + 5^{-1} + 1)
    """
    # If the input is of shape (N,), reshape it to (N, 1)
    if x.ndim == 1:
        x = x[:, None]
    elif x.ndim == 2 and x.shape[1] == 1:
        pass
    else:
        raise ValueError("Input must be of shape (N,) or (N, 1)")

    # Compute the function
    term1 = -(x - 1) ** 2
    term2 = np.sin(3 * x + 1 / 5 + 1)
    val = term1 * term2
    return val



# Define the objective function
def objective_function(x):
    return sinusoidal_synthetic(x)

# Generate input data (e.g., from -10 to 10)
x_values = np.linspace(-10, 10, 100).reshape(-1, 1)
y_values = objective_function(x_values)

# Create the plot
fig = go.Figure()

# Add trace for the objective function
fig.add_trace(go.Scatter(x=x_values.flatten(), y=y_values.flatten(), mode='lines', name='Objective Function'))

# Update layout
fig.update_layout(
    title="Objective Function Plot",
    xaxis_title="x",
    yaxis_title="Objective Function Value",
    showlegend=True
)

# Show the plot
fig.show()


In [5]:
import sys
sys.path.append('..')


import jax
import jax.numpy as jnp
import numpy as np
from gpax.acquisition import EI, optimize_acq
from gpax.models.gp import ExactGP
from jax import random

from src.utils_bo import DataTransformer, generate_initial_data


# Define the main function for running Bayesian Optimization
def run_bo(experiment_settings):
    # Extract settings from the dictionary
    search_space = experiment_settings["search_space"]
    num_iterations = experiment_settings["num_iterations"]
    objective_function = experiment_settings["objective_function"]

    # Acquisition and surrogate settings
    acq_settings = experiment_settings["acquisition"]
    surrogate_settings = experiment_settings["surrogate"]

    # Create the data transformer for normalization and standardization
    data_transformer = DataTransformer(search_space, surrogate_settings)

    # Step 1: Generate initial data using Sobol sequences
    X_init, y_init = generate_initial_data(
        objective_function, search_space, n=acq_settings["num_initial_guesses"]
    )

    # Normalize the search space bounds
    lb_normalized = data_transformer.normalize(
        search_space[0].reshape(1, search_space.shape[1])
    )
    ub_normalized = data_transformer.normalize(
        search_space[1].reshape(1, search_space.shape[1])
    )

    # Initialize history with the original (non-transformed) initial data
    X_history = X_init
    y_history = y_init

    # Apply transformations (normalization and standardization)
    X_normalized, y_standardized = data_transformer.apply_transformation(X_init, y_init)

    # Step 2: Initialize the GP model
    rng_key = random.PRNGKey(0)  # JAX random number generator
    gp_model = ExactGP(
        input_dim=search_space.shape[1], kernel=surrogate_settings["kernel"]
    )  # Using specified kernel
    gp_model.fit(rng_key, jnp.array(X_normalized), jnp.array(y_standardized))

    # Step 3: Main loop for Bayesian optimization
    for i in range(num_iterations):
        # Step 3.1: Optimize the acquisition function to find the next point
        X_next_normalized = optimize_acq(
            rng_key,
            gp_model,
            EI,
            num_initial_guesses=acq_settings["num_initial_guesses"],
            lower_bound=lb_normalized,  # In normalized space
            upper_bound=ub_normalized,  # In normalized space
            best_f=jnp.min(jnp.array(y_standardized)),
            maximize=acq_settings["maximize"],
            n=acq_settings["num_samples"],
        )

        X_next_normalized = X_next_normalized.reshape(1, search_space.shape[1])  # Ensure shape is (1, search_space.shape[1])
        
        print()
        print(f'x_next_normalized: {X_next_normalized}')

        # Step 3.2: Inverse-transform the input back to original space
        X_next = data_transformer.inverse_normalize(X_next_normalized)

        print(f'x_next: {X_next}')  
        print()

        # Step 3.3: Evaluate the objective function at the selected point
        y_next = objective_function(X_next)

        # Step 3.4: Add the new (non-transformed) data point to the history
        X_history = np.vstack((X_history, np.array(X_next)))
        y_history = np.vstack((y_history, np.array(y_next)))

        # Apply transformations to the updated dataset (for GP model fitting)
        X_transformed, y_transformed = data_transformer.apply_transformation(X_history, y_history)

        # Step 3.5: Re-train the GP model with the updated transformed dataset
        gp_model.fit(rng_key, jnp.array(X_transformed), jnp.array(y_transformed))

        # Step 3.6: Display progress
        print(f"Iteration {i + 1}: X_next = {X_next}, y_next = {y_next}")

    return X_history, y_history

In [7]:
objective_function = sinusoidal_synthetic

# Example experiment settings
experiment_settings = {
    "search_space": np.array([[-10], [10]]),  # 1D search space example
    "num_iterations": 10,  # Number of optimization iterations
    "objective_function": objective_function,  # Actual objective function
    "acquisition": {  # Acquisition function settings
        "num_samples": 10,  # Number of samples for acquisition function
        "num_initial_guesses": 10,  # Number of initial guesses for acquisition
        "maximize": False,  # Whether to maximize the acquisition function
    },
    "surrogate": {  # Surrogate model (GP) settings
        "kernel": "RBF",  # Kernel for GP model
        "normalize": True,  # Whether to normalize inputs
        "standardize": True,  # Whether to standardize outputs
    },
}

# Run the Bayesian optimization
X_history, y_history = run_bo(experiment_settings)

# Final result
print()
print()
print()
print()
print()
print("Final X history (in original space):", X_history)
print("Final y history:", y_history)

# The final result is in X_history and y_history, containing all evaluated points and their original function values.
optimal_index = np.argmin(y_history)
print(f"Optimal X: {X_history[optimal_index]}")
print(f"Optimal y: {y_history[optimal_index]}")

sample: 100%|██████████| 4000/4000 [00:01<00:00, 2848.01it/s, 3 steps of size 6.50e-01. acc. prob=0.81]



                 mean       std    median      5.0%     95.0%     n_eff     r_hat
k_length[0]      1.24      1.88      0.57      0.03      2.88    970.03      1.00
    k_scale      1.15      1.32      0.74      0.04      2.50   1117.88      1.00
      noise      1.12      0.72      0.97      0.17      2.01    853.72      1.00


x_next_normalized: [[0.56605595]]
x_next: [[1.3211193]]



sample: 100%|██████████| 4000/4000 [00:01<00:00, 2742.79it/s, 3 steps of size 5.54e-01. acc. prob=0.82]



                 mean       std    median      5.0%     95.0%     n_eff     r_hat
k_length[0]      1.05      1.75      0.43      0.05      2.45    796.55      1.00
    k_scale      1.29      1.45      0.83      0.06      2.80    834.29      1.00
      noise      0.98      0.66      0.83      0.15      1.80    879.82      1.00

Iteration 1: X_next = [[1.3211193]], y_next = [[0.09280846]]

x_next_normalized: [[0.5599238]]
x_next: [[1.1984768]]



sample: 100%|██████████| 4000/4000 [00:01<00:00, 2549.00it/s, 3 steps of size 3.55e-01. acc. prob=0.93] 


...


                 mean       std    median      5.0%     95.0%     n_eff     r_hat
k_length[0]      0.25      0.20      0.21      0.08      0.43    743.79      1.00
    k_scale      2.60      2.15      1.96      0.31      5.22    862.22      1.00
      noise      0.30      0.20      0.25      0.06      0.52    727.23      1.00

Iteration 8: X_next = [[1.9979439]], y_next = [[-0.7866552]]

x_next_normalized: [[0.64714265]]
x_next: [[2.942853]]



sample: 100%|██████████| 4000/4000 [00:01<00:00, 2250.79it/s, 3 steps of size 4.76e-01. acc. prob=0.93]



                 mean       std    median      5.0%     95.0%     n_eff     r_hat
k_length[0]      0.25      0.18      0.22      0.07      0.41    686.27      1.00
    k_scale      2.80      2.44      2.11      0.38      5.66    992.77      1.00
      noise      0.28      0.19      0.23      0.08      0.48    576.83      1.00

Iteration 9: X_next = [[2.942853]], y_next = [[2.1431065]]

x_next_normalized: [[0.47168505]]
x_next: [[-0.56629944]]



sample: 100%|██████████| 4000/4000 [00:01<00:00, 2163.89it/s, 3 steps of size 5.60e-01. acc. prob=0.88]


                 mean       std    median      5.0%     95.0%     n_eff     r_hat
k_length[0]      0.24      0.12      0.21      0.09      0.41    974.56      1.00
    k_scale      2.90      2.44      2.15      0.39      5.99    884.11      1.00
      noise      0.25      0.13      0.22      0.09      0.43    912.80      1.00

Iteration 10: X_next = [[-0.56629944]], y_next = [[1.173799]]





Final X history (in original space): [[ 4.49151982]
 [-7.58229332]
 [-2.70112146]
 [ 9.60065803]
 [ 5.4327099 ]
 [-2.34176656]
 [-7.37522177]
 [ 0.47585385]
 [ 1.32111931]
 [ 1.19847679]
 [ 1.2220335 ]
 [ 1.32453346]
 [ 2.1204071 ]
 [ 1.06153488]
 [ 2.58002663]
 [ 1.99794388]
 [ 2.94285297]
 [-0.56629944]]
Final y history: [[-1.04723857e+01]
 [ 3.16570689e+01]
 [ 7.96119017e+00]
 [ 7.30633350e+01]
 [ 1.91780252e+01]
 [-4.93657898e+00]
 [ 6.13779081e+01]
 [-1.35081978e-01]
 [ 9.28084552e-02]
 [ 3.92572917e-02]
 [ 4.87176254e-02]
 [ 9.43173096e-02]
 [-1.20189965e+00]
 [ 3.58493836e-03]
 [-1.1632152